In [1]:
#Now we analysing the sentiment of the book descriptions, we had the choice of fine tuning our own model or using a fine tuned model. we are using a fine tuned model
import pandas as pd
books = pd.read_csv('books_with_categories.csv')

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None)
classifier("YOuSuck!!")  #just chiecking the model

Device set to use cpu


[[{'label': 'disgust', 'score': 0.750728189945221},
  {'label': 'anger', 'score': 0.09209901839494705},
  {'label': 'sadness', 'score': 0.08928407728672028},
  {'label': 'surprise', 'score': 0.039683643728494644},
  {'label': 'neutral', 'score': 0.01503510307520628},
  {'label': 'fear', 'score': 0.00967804342508316},
  {'label': 'joy', 'score': 0.003491829615086317}]]

In [4]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [5]:
classifier(books['description'][0]) #analyses the entire description

[[{'label': 'fear', 'score': 0.6548418998718262},
  {'label': 'neutral', 'score': 0.16985182464122772},
  {'label': 'sadness', 'score': 0.11640854179859161},
  {'label': 'surprise', 'score': 0.020700592547655106},
  {'label': 'disgust', 'score': 0.01910073682665825},
  {'label': 'joy', 'score': 0.015161239542067051},
  {'label': 'anger', 'score': 0.00393515033647418}]]

In [6]:
classifier(books['description'][0].split('.')) #analyses each sentence of the description

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.14038576185703278},
  {'label': 'fear', 'score': 0.06816209107637405},
  {'label': 'joy', 'score': 0.04794240742921829},
  {'label': 'anger', 'score': 0.00915635284036398},
  {'label': 'disgust', 'score': 0.0026284719351679087},
  {'label': 'sadness', 'score': 0.0021221607457846403}],
 [{'label': 'neutral', 'score': 0.449370414018631},
  {'label': 'disgust', 'score': 0.27359241247177124},
  {'label': 'joy', 'score': 0.10908260941505432},
  {'label': 'sadness', 'score': 0.09362703561782837},
  {'label': 'anger', 'score': 0.040478307753801346},
  {'label': 'surprise', 'score': 0.02697017230093479},
  {'label': 'fear', 'score': 0.006879065651446581}],
 [{'label': 'neutral', 'score': 0.6462168097496033},
  {'label': 'sadness', 'score': 0.2427326887845993},
  {'label': 'disgust', 'score': 0.04342268034815788},
  {'label': 'surprise', 'score': 0.028300466015934944},
  {'label': 'joy', 'score': 0.014211482

In [10]:
#but obviously anaysing each sentence will complicate things and but it will be more accurate
# so we have to do a lot of processing to correctly give the sentiment output
# the idea is to show each sentiment's highest probability for the full description
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [11]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [12]:
    emotion_scores

{'anger': [np.float64(0.06413363665342331),
  np.float64(0.6126185059547424),
  np.float64(0.06413363665342331),
  np.float64(0.3514834940433502),
  np.float64(0.0814124047756195),
  np.float64(0.23222540318965912),
  np.float64(0.5381839871406555),
  np.float64(0.06413363665342331),
  np.float64(0.30066895484924316),
  np.float64(0.06413363665342331)],
 'disgust': [np.float64(0.27359241247177124),
  np.float64(0.3482835590839386),
  np.float64(0.10400672256946564),
  np.float64(0.15072275698184967),
  np.float64(0.18449535965919495),
  np.float64(0.7271743416786194),
  np.float64(0.1558549404144287),
  np.float64(0.10400672256946564),
  np.float64(0.27948135137557983),
  np.float64(0.17792679369449615)],
 'fear': [np.float64(0.928168535232544),
  np.float64(0.942527711391449),
  np.float64(0.9723208546638489),
  np.float64(0.36070650815963745),
  np.float64(0.0950433686375618),
  np.float64(0.05136279761791229),
  np.float64(0.7474278807640076),
  np.float64(0.40449780225753784),
  np

In [14]:
#we run it for all books now

from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5088/5088 [12:32<00:00,  6.76it/s]


In [15]:
#create a df for an emotion scores
emotion_df = pd.DataFrame(emotion_scores)
emotion_df["isbn13"] = isbn
emotion_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273592,0.928169,0.932798,0.646217,0.967158,0.729603,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5083,0.148208,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656,9788172235222
5084,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5085,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625,9788179921623
5086,0.064134,0.104007,0.459268,0.759456,0.951104,0.368112,0.078765,9788185300535


In [16]:
#now merge the emotion_df with the original nooks df

books = pd.merge(books ,emotion_df, on="isbn13")

In [17]:
books.to_csv("books_with_emotions.csv", index=False)

In [18]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,simple_categories,predicted_categories_x,predicted_categories_y,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Fiction,Fiction,Fiction,0.064134,0.273592,0.928169,0.932798,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Fiction,Fiction,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Fiction,Fiction,Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,Nonfiction,Nonfiction,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,Nonfiction,Nonfiction,Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Fiction,Fiction,Fiction,0.148208,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656
5084,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Nonfiction,Nonfiction,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5085,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,Fiction,Fiction,Fiction,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625
5086,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,Nonfiction,NaN,NaN,0.064134,0.104007,0.459268,0.759456,0.951104,0.368112,0.078765
